REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [2]:
import pandas as pd

In [231]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [232]:
bus.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [233]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [6]:
len(bus)

6315

#### Question 2b: How many unique business IDs are there?  

In [6]:
##business ids are unique, also 6315 business IDs
len(bus['business_id'].unique())

6315

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [32]:
##5 most common by name
sub_bus = bus[['name', 'state']].rename(
    columns={'state': 'count_name'}).groupby(['name']).count()
sub_bus.sort_values('count_name', ascending=False).head(5)

,count_name
name,
STARBUCKS COFFEE,72
PEET'S COFFEE & TEA,24
MCDONALDS,12
WALGREENS,11
SAN FRANCISCO SOUP COMPANY,11


In [34]:
sfbus = bus[bus['city'] == 'San Francisco']
##All of the most common are in San Francisco, same counts
sub_sfbus = sfbus[['name', 'city','state']].rename(
    columns={'state': 'count_name'}).groupby(['name','city']).count()
sub_sfbus.sort_values('count_name', ascending=False).head(5)

,,count_name
name,city,
STARBUCKS COFFEE,San Francisco,72
PEET'S COFFEE & TEA,San Francisco,24
MCDONALDS,San Francisco,12
WALGREENS,San Francisco,11
SAN FRANCISCO SOUP COMPANY,San Francisco,11


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [29]:
##Zip codes are stored as an object data type
bus.dtypes

business_id       int64
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
phone_number     object
dtype: object

In [30]:
bus['postal_code'].dtypes

dtype('O')

#### Question 3b

What are the unique values of postal_code?

In [36]:
##47 unique values, with some not valid as a zip codes (CA, Ca, 00000, null value)
##len(bus['postal_code'].unique())
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [38]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [39]:
##5999 entries with valid zip codes
vb = bus[bus['postal_code'].isin(validZip)]
len(vb)

5999

## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [166]:
vb[pd.isnull(vb['longitude'])]
##4a. 2483 rows/entries missing longitude values

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482
...,...,...,...,...,...,...,...,...,...
6310,90234,ACCESS SFUSD THE ARC,1520 HOWARD ST,San Francisco,CA,94103,NaN,NaN,NaN
6311,90236,PHO DONG HUONG,667 MONTEREY BLVD,San Francisco,CA,94127,NaN,NaN,+14154945048
6312,90237,WIN'S RESTAURANT,3040 TARAVAL ST,San Francisco,CA,94116,NaN,NaN,+14158606036
6313,90238,USA POWER MARKET,1524 OCEAN AVE,San Francisco,CA,94112,NaN,NaN,+14153331878


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [169]:
##use valid zip and join grouped count of null_lon and not_null_log
leftzip = vb[['postal_code', 'state']].rename(columns={'state': 'numinzip'}).groupby(['postal_code']).count()

long = vb[vb['longitude'].isnull()].groupby(['postal_code', 'state']).count().reset_index()
nanlong = long[['postal_code', 'business_id']].rename(columns={'business_id': 'null_long'}).set_index('postal_code')

lat = vb[vb['latitude'].notnull()].groupby(['postal_code', 'state']).count().reset_index()
notlat = lat[['postal_code', 'business_id']].rename(columns={'business_id': 'not_null_long'}).set_index('postal_code')

In [175]:
newleftzip = leftzip.join(nanlong)
df_zipcode = newleftzip.join(notlat)
df_zipcode

,numinzip,null_long,not_null_long
postal_code,,,
94102,458,196,262
94103,558,252,306
94104,133,59,74
94105,226,100,126
94107,451,247,204
94108,224,82,142
94109,383,146,237
94110,585,251,334
94111,282,115,167


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [182]:
df_zipcode['per_missinglong'] = round(df_zipcode['null_long']/df_zipcode['numinzip']*100,2)
df_zipcode.sort_values('prop_long', ascending=False).head(5)

,numinzip,null_long,not_null_long,prop_long,per_missinglong
postal_code,,,,,
94107,451,247,204,54.77,54.77
94124,164,88,76,53.66,53.66
94132,134,65,69,48.51,48.51
94114,230,108,122,46.96,46.96
94122,270,123,147,45.56,45.56


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [75]:
#clearly not.
all_id = len(ins['business_id'])
unique_id = len(ins['business_id'].unique())
print("There are {0} business IDs and {1} unique business IDs".format(all_id, unique_id))

There are 15430 business IDs and 5730 unique business IDs


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [234]:
#ins.groupby(['type']).count()
ins['type'].value_counts()

#takes on Routine and Complaint

routine      15429
complaint        1
Name: type, dtype: int64

In [235]:
#filter out complaint as it only occurs once
ins2 = ins[ins['type'] == 'routine'].groupby(['type']).count()
ins2

,business_id,score,date
type,,,
routine,15429,15429,15429


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [282]:
ins['date_dt'] = pd.to_datetime(ins['date'], format= '%Y%m%d')
ins['year'] = ins['date_dt'].dt.year
ins

,business_id,score,date,type,date_dt,year
0,10,82,20160503,routine,2016-05-03,2016
1,10,94,20140729,routine,2014-07-29,2014
2,10,92,20140114,routine,2014-01-14,2014
3,19,94,20160513,routine,2016-05-13,2016
4,19,94,20141110,routine,2014-11-10,2014
...,...,...,...,...,...,...
15425,89515,98,20161103,routine,2016-11-03,2016
15426,89701,100,20161206,routine,2016-12-06,2016
15427,89790,96,20161129,routine,2016-11-29,2016
15428,89803,100,20161215,routine,2016-12-15,2016


In [283]:
ins.groupby(['year']).count()

,business_id,score,date,type,date_dt
year,,,,,
2013,38,38,38,38,38
2014,5629,5629,5629,5629,5629
2015,3923,3923,3923,3923,3923
2016,5840,5840,5840,5840,5840


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [258]:
##Range of years 2013 - 2016. 2013 only has 38 entries, will filter for greater than or = to 2014 
ins3 = ins[(ins['year'] >= 2014)].rename(columns={'score': 'count_ins'}).groupby(['year']).count().reset_index()
ins3[['year','count_ins']]

,year,count_ins
0,2014,5629
1,2015,3923
2,2016,5840


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [269]:
ins2016 = ins[(ins['year'] == 2016)]
ins2016

,business_id,score,date,type,date_dt,year
0,10,82,20160503,routine,2016-05-03,2016
3,19,94,20160513,routine,2016-05-13,2016
6,24,98,20161005,routine,2016-10-05,2016
7,24,96,20160311,routine,2016-03-11,2016
10,31,84,20160816,routine,2016-08-16,2016
...,...,...,...,...,...,...
15425,89515,98,20161103,routine,2016-11-03,2016
15426,89701,100,20161206,routine,2016-12-06,2016
15427,89790,96,20161129,routine,2016-11-29,2016
15428,89803,100,20161215,routine,2016-12-15,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [275]:
buswithins = pd.merge(bus, ins2016, on="business_id")
buswithins.head(4)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,date_dt,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82,20160503,routine,2016-05-03,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94,20160513,routine,2016-05-13,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98,20161005,routine,2016-10-05,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96,20160311,routine,2016-03-11,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [278]:
buswithins[['name', 'address','score']].sort_values('score', ascending=True).set_index('score').head(20)

,name,address
score,,
52,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD
55,POKI TIME,2101 LOMBARD ST
55,CRAZY PEPPER,2257 SAN JOSE AVE
55,UNCLE CAFE,65 WAVERLY PL
56,HING WANG BAKERY,339 JUDAH ST
56,GOLDEN WOK,295 B ORIZABA AVE
57,TOP'S CAFE,2460 SAN BRUNO AVE
57,L & G VIETNAMESE SANDWICH,602 EDDY ST
57,RED A BAKERY,634 CLEMENT ST


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!